In [1]:
import os
import pandas
import numpy as np
import pyvisa
import time
import datetime
import ipywidgets as iw
from IPython.display import display
from matplotlib import pyplot as plt

In [2]:
#--- Obtaining the VISA addresses of connected devices ---
rm = pyvisa.ResourceManager()
print(f"VISA addresses of connected devices are as below:\n{rm.list_resources()}")

VISA addresses of connected devices are as below:
('USB0::0x0957::0x179A::MY62160598::INSTR', 'USB0::0x0699::0x034C::C010132::INSTR', 'ASRL1::INSTR')


In [3]:
print(rm.visalib)

Visa Library at C:\WINDOWS\system32\visa32.dll


In [35]:
#--- Defining device name (COPY VISA ADDRESSES & EDIT) ---
scope = rm.open_resource('USB0::0x0957::0x179A::MY62160598::INSTR', timeout=1000)
afg = rm.open_resource('USB0::0x0699::0x034C::C010132::INSTR', timeout=1000)

In [36]:
#--- Confirm oscilloscope ID ---
scope_id = scope.query("*IDN?")
afg_id = afg.query("*IDN?")
print(f"scope ID = {scope_id}")
print(f"function generator ID = {afg_id}")

scope ID = AGILENT TECHNOLOGIES,DSO-X 2004A,MY62160598,02.65.2021030741

function generator ID = TEKTRONIX,AFG3102C,C010132,SCPI:99.0 FV:1.0.2



In [25]:
scope.write(":WAV:MODE NORM")
scope.write(":WAV:FORM ASCii")

17

In [26]:
freq = 10 #MHz
scope.write(f":TIMEBASE:SCALE {5e-6/freq}")

39

In [27]:
offset = 0.4 #V
scope.write(":TRIG:MODE EDGE")
scope.write(":TRIG:EDGE:SOUR CHAN1")
scope.write(":TRIG:EDGE:SLOPE POSITIVE")
scope.write(f":TRIG:LEVEL {offset}")

17

In [29]:
scope_dt = float(scope.query(":WAV:XINC?"))
scope_dd = float(scope.query(":WAV:YINC?"))
scope_ot = float(scope.query(":WAV:XOR?"))
scope_points = int(scope.query(":WAV:POINTs?"))

print(scope_dt)
print(scope_dd)
print(scope_ot)
print(scope_points)

1e-09
3.140703e-05
-2.5e-06
5000


In [41]:
pulse_width = 30
amp = 3.0
phase = 45
#--- Function Generator Output Setup ---
afg.write("FUNC:SHAP PULS")
afg.write(f"FREQ {freq}E6")
afg.write(f"PULS:{pulse_width}")
afg.write(f"VOLT {amp}")
afg.write(f"VOLT:OFFS {offset}")
afg.write(f"PULSE: DELAY {phase}")
afg.write("OUTP:IMP 50")

13

In [60]:
afg.write(f"FREQ 5E6")
afg.write("OUTP ON")

9

In [61]:
afg.write("OUTP OFF")

10

In [59]:
afg.write("FUNC:SHAP PULS")
afg.write(f"FREQ {freq}E6")
afg.write(f"PULS:{pulse_width}")
afg.write(f"VOLT {amp}")
afg.write(f"VOLT:OFFS {offset}")
afg.write(f"PULSE: DELAY {phase}")
afg.write("OUTP:IMP 50")

13

In [51]:
afg.write("SOUR2: FUNC PULS")

18